In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
DATAPATH = "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer/data/"

# Check the timeseries data and collect the patient ID 

In [ ]:
lvl2_train_imputer = pd.read_pickle(os.path.join(DATAPATH, "lvl2_imputer_train.pkl"))
lvl2_dev_imputer = pd.read_pickle(os.path.join(DATAPATH, "lvl2_imputer_dev.pkl"))
lvl2_test_imputer = pd.read_pickle(os.path.join(DATAPATH,"lvl2_imputer_test.pkl"))
Ys = pd.read_pickle(os.path.join(DATAPATH, "Ys.pkl"))

In [ ]:
print("Shape of train, dev, test {}, {}, {}.".format((lvl2_train_imputer.shape), (lvl2_dev_imputer.shape), (lvl2_test_imputer.shape)))
print( 402120 / 24, 57432 / 24, 114936 / 24)

In [ ]:
patient_ids = [] # store all patient ids 
for each_entry in Ys.index:
    patient_ids.append(each_entry[0])

In [ ]:
print("Number of total patient {}".format(len(patient_ids)))

# Select Clinical Notes

In [ ]:
admission_df = pd.read_csv(os.path.join(DATAPATH, "ADMISSIONS.csv"))
noteevents_df = pd.read_csv(os.path.join(DATAPATH, "NOTEEVENTS.csv"))
icustays_df = pd.read_csv(os.path.join(DATAPATH, "ICUSTAYS.csv"))

In [ ]:
noteevents_df.groupby(noteevents_df.CATEGORY).agg(['count'])

In [ ]:
note_categories = noteevents_df.groupby(noteevents_df.CATEGORY).agg(['count']).index

In [ ]:
#selected_note_types = ['Nursing/other', 'Radiology', 'Nursing', 'ECG', 'Physician', 'Echo', 'Respiratory', 'Nutrition']
selected_note_types = []
for each_cat in list(note_categories):
    if each_cat != 'Discharge summary':
        selected_note_types.append(each_cat)

In [ ]:
noteevents_df.shape

### Select based on note category

In [ ]:
sub_notes = noteevents_df[noteevents_df.CATEGORY.isin(selected_note_types)]

In [ ]:
sub_notes.shape

### Drop no chart notes

In [ ]:
missing_chardate_index = []
for each_note in sub_notes.itertuples():
    if isinstance(each_note.CHARTTIME, str):
        continue
    if np.isnan(each_note.CHARTTIME):
        missing_chardate_index.append(each_note.Index)
print ("{} of notes does not charttime.".format(len(missing_chardate_index)))
print(sub_notes.shape)

In [ ]:
sub_notes.drop(missing_chardate_index, inplace=True )
print(sub_notes.shape)

### Select based on patient id

In [ ]:
sub_notes = sub_notes[sub_notes.SUBJECT_ID.isin(patient_ids)] ## select based on patient id

In [ ]:
sub_notes.shape

### Select based on time limit (24 hours) 

In [ ]:
MIMIC_EXTRACT_DATA = DATAPATH + "all_hourly_data.h5"
stats = pd.read_hdf(MIMIC_EXTRACT_DATA, 'patients')
TIMELIMIT = 1 # 1day

In [ ]:
stats.shape

In [ ]:
stats.head()

In [ ]:
new_stats = stats.reset_index()

In [ ]:
new_stats.rename(columns = {"subject_id": "SUBJECT_ID", "hadm_id": "HADM_ID"}, inplace=True)

In [ ]:
print(new_stats.shape, sub_notes.shape)

In [ ]:
df_adm_notes = pd.merge(sub_notes[['ROW_ID','SUBJECT_ID','HADM_ID','CHARTTIME', 'CATEGORY', 'TEXT']],
                        new_stats[['SUBJECT_ID','HADM_ID','icustay_id','age','admittime','dischtime', 'deathtime', 
                                  'intime', 'outtime', 'los_icu', 'mort_icu', 'mort_hosp', 'hospital_expire_flag',
                                  'hospstay_seq', 'max_hours']], 
                        on = ['SUBJECT_ID'],
                        how = 'left')

In [ ]:
df_adm_notes.head()

In [ ]:
df_adm_notes['CHARTTIME'] = pd.to_datetime(df_adm_notes['CHARTTIME'])

In [ ]:
df_less_n = df_adm_notes[((df_adm_notes['CHARTTIME']-df_adm_notes['intime']).dt.total_seconds()/(24*60*60))<TIMELIMIT]

In [ ]:
df_less_n.shape

# Save clinical notes

In [ ]:
pd.to_pickle(df_less_n, (os.path.join(DATAPATH, "sub_notes.p")))